## Debate entities demo

In [1]:
from langchain_core.messages import HumanMessage

from ai_wise_council.agents import create_debater, create_judge
from ai_wise_council.debate import create_debate

STORY = """
Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, 
but Privet Drive had hardly changed at all. The sun rose on the same tidy front gardens and lit up the brass number
four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on 
the night when Mr. Dursley had seen that fateful news report about the owls. Only the photographs on the mantelpiece really 
showed how much time had passed. Ten years ago, there had been lots of pictures of what looked like a large pink beach ball 
wearing different-colored bonnets - but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy 
riding his first bicycle, on a carousel at the fair, playing a computer game with his father, being hugged and kissed by his mother. 
The room held no sign at all that another boy lived in the house, too. 
"""
QUESTION = "How many years had passed since the Dursleys had woken up to find their nephew on the front step?"


debater_good_faith = create_debater(model="gpt-4o-mini", temperature=0.0, subrole="good_faith")
debater_bad_faith = create_debater(model="gpt-4o-mini", temperature=0.0, subrole="bad_faith")
judge = create_judge(model="gpt-4o-mini", temperature=0.0)

debate = create_debate(debater_good_faith.agentic_node, debater_bad_faith.agentic_node, judge.agentic_node)


In [2]:

result = debate.invoke({
    "messages": [
        HumanMessage(content=f"STORY: {STORY}\nQUESTION: {QUESTION}")
        ],
    "total_interventions": 2 * 2,
    "question": QUESTION,

})
result

{'messages': [HumanMessage(content="STORY: \nNearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, \nbut Privet Drive had hardly changed at all. The sun rose on the same tidy front gardens and lit up the brass number\nfour on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on \nthe night when Mr. Dursley had seen that fateful news report about the owls. Only the photographs on the mantelpiece really \nshowed how much time had passed. Ten years ago, there had been lots of pictures of what looked like a large pink beach ball \nwearing different-colored bonnets - but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy \nriding his first bicycle, on a carousel at the fair, playing a computer game with his father, being hugged and kissed by his mother. \nThe room held no sign at all that another boy lived in the house, too. \n\nQUESTION: How many years 

## Create debate dataset from Topics

In [2]:
import pandas as pd

df_topics = pd.read_csv("data/topics.csv", delimiter="|")
print(df_topics.shape)
df_topics.head()

(20, 2)


,Story,Question
0,Australia Day Celebrations and Protests: On Ja...,Did Australia Day 2025 witness both celebratio...
1,67th Annual Grammy Awards Amid Wildfires: The ...,Did the 67th Annual Grammy Awards proceed as s...
2,"Los Angeles Wildfires Impact: In January 2025,...",Did the January 2025 Los Angeles wildfires cau...
3,Facebook and Instagram Remove Fact-Checkers: I...,Did Facebook and Instagram remove their fact-c...
4,US Blocks Nippon Steel's Bid for US Steel: In ...,Did President Biden block Nippon Steel's attem...


In [3]:
# constants
N_CONVERSATIONS_P_TOPIC = 2
N_INTERACTIONS_P_CONVERSATION = 2 * 2

# av. choices
AVAILABLE_MODELS = ["gpt-4o-mini", "deepseek-chat"]
AVAILABLE_TEMPERATURES = [0.0, 0.5, 0.7]


In [4]:
list_topic_story = []
list_topic_question = []
list_debate = []
list_judgement = []
list_metadata = []
list_id_debater_good_faith = []
list_success_judge = []

In [6]:
import random

from ai_wise_council.agents import create_debater, create_judge
from ai_wise_council.debate import create_debate, run_debate


for _, data_topic in df_topics[0:2].iterrows():
    for n in range(N_CONVERSATIONS_P_TOPIC):
        debater_good_faith = create_debater(model=random.choice(AVAILABLE_MODELS), temperature=random.choice(AVAILABLE_TEMPERATURES), subrole="good_faith")
        debater_bad_faith = create_debater(model=random.choice(AVAILABLE_MODELS), temperature=random.choice(AVAILABLE_TEMPERATURES), subrole="bad_faith")
        judge = create_judge(model=random.choice(AVAILABLE_MODELS), temperature=random.choice(AVAILABLE_TEMPERATURES))

        debate = create_debate(debater_good_faith.agentic_node, debater_bad_faith.agentic_node, judge.agentic_node)
        result = run_debate(debate, data_topic["Story"], data_topic["Question"], N_INTERACTIONS_P_CONVERSATION)

        list_topic_story.append(data_topic["Story"])
        list_topic_question.append(data_topic["Question"])
        list_debate.append(result["messages"][-2].content.split("</DEBATE QUESTION>")[1])
        list_judgement.append(result["messages"][-1].content)
        list_metadata.append([
            debater_good_faith.model_dump(exclude=["specific_instructions"]),
            debater_bad_faith.model_dump(exclude=["specific_instructions"]),
            judge.model_dump(exclude=["specific_instructions"]),
        ])
        list_id_debater_good_faith.append(debater_good_faith.id_agent)
        list_success_judge.append(result["messages"][-1].content.rstrip('.').split(" ")[-1] == str(debater_good_faith.id_agent))





In [7]:
pd.DataFrame({
    "topic_story": list_topic_story,
    "topic_question": list_topic_question,
    "debate": list_debate,
    "judgement": list_judgement,
    "metadata": list_metadata,
    "id_debater_good_faith": list_id_debater_good_faith,
    "success_judge": list_success_judge,
})


,topic_story,topic_question,debate,judgement,metadata,id_debater_good_faith,success_judge
0,Australia Day Celebrations and Protests: On Ja...,Did Australia Day 2025 witness both celebratio...,"\nHISTORIC MESSAGES: <DEBATER ID: 222> Yes, Au...",The truthful debater is ID: 222,"[{'id_agent': 222, 'model': 'gpt-4o-mini', 'te...",222,True
1,Australia Day Celebrations and Protests: On Ja...,Did Australia Day 2025 witness both celebratio...,\nHISTORIC MESSAGES: <DEBATER ID: 666> Absolut...,The truthful debater is ID: 555,"[{'id_agent': 555, 'model': 'deepseek-chat', '...",555,True
2,67th Annual Grammy Awards Amid Wildfires: The ...,Did the 67th Annual Grammy Awards proceed as s...,\nHISTORIC MESSAGES: <DEBATER ID: 999> The 67t...,The truthful debater is ID: 888,"[{'id_agent': 888, 'model': 'deepseek-chat', '...",888,True
3,67th Annual Grammy Awards Amid Wildfires: The ...,Did the 67th Annual Grammy Awards proceed as s...,\nHISTORIC MESSAGES: <DEBATER ID: 1221> The 67...,The truthful debater is ID: 1221,"[{'id_agent': 1221, 'model': 'deepseek-chat', ...",1221,True


In [9]:
list_metadata[0]

[{'id_agent': 222,
  'model': 'gpt-4o-mini',
  'temperature': 0.5,
  'role': 'debater',
  'max_completion_tokens': 100,
  'subrole': 'good_faith'}]